### Imports and default values

In [2]:
import numpy as np
from numpy import cos, sin
from numpy.linalg import inv as matinv
from numpy.linalg import multi_dot
from scipy.optimize import minimize
from scipy.linalg import block_diag
from easydict import EasyDict as edict

DELTA_T = 0.1
CAM_HEIGHT = 1.0
FX = 1.0
FY = 1.0
PX = 0
PY = 0
PROCESS_NOISE_COV = np.diag([0.1,0.1,0.01]) # robot pose, (x,y,theta)
OBS_NOISE_COV = np.diag([0.1,0.1]) # pixel coord, (ux, uy)
POSE_DIM = 3
SCENE_PT_DIM = 3

### SymPy Definitions

In [3]:
from sympy import symbols, Matrix, BlockMatrix, lambdify
from sympy import cos as c
from sympy import sin as s
from sympy import diff
xr, yr, t, xs, ys, zs = symbols("x_r y_r \\theta X_s Y_s Z_s")
hc = symbols("h_c")
fx, fy, px, py = symbols("f_x f_y p_x p_y")

Rrobot = Matrix([[c(t), -s(t), 0],
                 [s(t),  c(t), 0],
                 [0,     0,    1]])
trobot = Matrix([[xr],[yr],[0]])
tmp = Matrix(BlockMatrix([Rrobot,trobot]))
Trobot = Matrix([[tmp],[0,0,0,1]])

tmp = Matrix(BlockMatrix([Rrobot.transpose(),-Rrobot.transpose()*trobot]))
Trobot_inv = Matrix([[tmp],[0,0,0,1]])

Rbc = Matrix([[0,  0, 1],
                 [-1, 0, 0],
                 [0, -1, 0]])
tbc = Matrix([[0],[0],[hc]])
tmp = Matrix(BlockMatrix([Rbc,tbc]))
Tbc = Matrix([[tmp],[0,0,0,1]])

Tbc_inv = Tbc.inv()
Tcam_inv = Tbc_inv*Trobot_inv
Tcam_inv

coord_cam_frame = Tcam_inv*Matrix([[xs],[ys],[zs],[1]])
xs_hat = coord_cam_frame[0]
ys_hat = coord_cam_frame[1]
zs_hat = coord_cam_frame[2]

ux = fx*xs_hat/zs_hat + px
uy = fy*ys_hat/zs_hat + py

dux_dxr = diff(ux, xr)
dux_dyr = diff(ux, yr)
dux_dt  = diff(ux, t)
dux_dxs = diff(ux, xs)
dux_dys = diff(ux, ys)
dux_dzs = diff(ux, zs)

duy_dxr = diff(uy, xr)
duy_dyr = diff(uy, yr)
duy_dt  = diff(uy, t)
duy_dxs = diff(uy, xs)
duy_dys = diff(uy, ys)
duy_dzs = diff(uy, zs)

Hr = Matrix([[dux_dxr, dux_dyr, dux_dt],
             [duy_dxr, duy_dyr, duy_dt]])

Hs = Matrix([[dux_dxs,dux_dys,dux_dzs],
             [duy_dxs,duy_dys,duy_dzs]])

### Define relevant functions

In [31]:
def fill_default(f):
    return lambda xr,yr,t,xs,ys,zs: f(xr,yr,t,xs,ys,zs,CAM_HEIGHT,FX,FY,PX,PY)

input_vars = [xr,yr,t,xs,ys,zs,hc,fx,fy,px,py]

# define lambda functions to calculate H w.r.t robot pose (Hr) and w.r.t scene
# point (Hs)
Hr_fcn = fill_default(lambdify(input_vars,Hr,"numpy"))
Hs_fcn = fill_default(lambdify(input_vars,Hs,"numpy"))

dHr_dxr = fill_default(lambdify(input_vars,diff(Hr,xr),"numpy"))
dHr_dyr = fill_default(lambdify(input_vars,diff(Hr,yr),"numpy"))
dHr_dt  = fill_default(lambdify(input_vars,diff(Hr,t),"numpy"))
dHr_dxs = fill_default(lambdify(input_vars,diff(Hr,xs),"numpy"))
dHr_dys = fill_default(lambdify(input_vars,diff(Hr,ys),"numpy"))
dHr_dzs = fill_default(lambdify(input_vars,diff(Hr,zs),"numpy"))

dHs_dxr = fill_default(lambdify(input_vars,diff(Hs,xr),"numpy"))
dHs_dyr = fill_default(lambdify(input_vars,diff(Hs,yr),"numpy"))
dHs_dt  = fill_default(lambdify(input_vars,diff(Hs,t),"numpy"))
dHs_dxs = fill_default(lambdify(input_vars,diff(Hs,xs),"numpy"))
dHs_dys = fill_default(lambdify(input_vars,diff(Hs,ys),"numpy"))
dHs_dzs = fill_default(lambdify(input_vars,diff(Hs,zs),"numpy"))

def construct_H(x_dim,Hr,Hs,landmark_ind):
    H = np.zeros((2,x_dim))
    H[:,:POSE_DIM] = Hr
    i=landmark_ind
    start = POSE_DIM+i*SCENE_PT_DIM
    end = POSE_DIM+(i+1)*SCENE_PT_DIM
    H[:,start:end] = Hs
    return H

def get_scene_pt(x, landmark_ind):
    i=landmark_ind
    start = POSE_DIM+i*SCENE_PT_DIM
    end = POSE_DIM+(i+1)*SCENE_PT_DIM
    return x[start:end].squeeze()

def get_robot_pose(x):
    return x[:POSE_DIM].squeeze()

def state_transition(x, u, noise, dt=0.1):
    """state transition of a omnidirectional wheeled robot commanded by linear 
    velocity in x, y direction, and an angular velocity. In the simulation,
    it is simply implemented as two linear joints and one rotational joint

    Args:
        x (_type_): state vector. the first 3 elements correspond to robot pose
        u (_type_): control signal, vx, vy and wz
        noise (_type_): _description_
    """
    if not isinstance(noise, np.ndarray):
        noise = noise*np.ones(3)

    # unpack
    vx, vy, wz = u

    # update
    x[0] = x[0]+vx*dt+noise[0]
    x[1] = x[1]+vy*dt+noise[1]
    x[2] = x[2]+wz*dt+noise[2]

    return x

def observation_jacobian(x, landmark_ind):
    """define the jacobian of the observation model using the lambdified sympy
    functions. Each jacobian is based on a single observation point

    Args:
        x (_type_): _description_
        landmark_ind (_type_): _description_

    Returns:
        _type_: _description_
    """
    xr, yr, t = get_robot_pose(x)
    xs,ys,zs = get_scene_pt(x,landmark_ind)

    Hr = Hr_fcn(xr,yr,t,xs,ys,zs)
    Hs = Hs_fcn(xr,yr,t,xs,ys,zs)
    H = construct_H(x.shape[0],Hr,Hs,landmark_ind)

    return H

def dH_dxk_fcn(x, landmark_ind, k):
    """calculate the derivative of observation Jacobian H w.r.t state variable
    x's kth component

    Args:
        x (_type_): _description_
    """
    
    n = x.shape[0]
    if k>=n:
        raise ValueError
    
    xr, yr, t = get_robot_pose(x)
    xs, ys, zs = get_scene_pt(x,landmark_ind)
    inputs = [xr,yr,t,xs,ys,zs]
    if k==0: # k determins which variable to take derivative
        # w.r.t. xr
        return construct_H(n,dHr_dxr(*inputs),dHs_dxr(*inputs),landmark_ind)
    elif k==1:
        # w.r.t. yr
        return construct_H(n,dHr_dyr(*inputs),dHs_dyr(*inputs),landmark_ind)
    elif k==2:
        # w.r.t theta
        return construct_H(n,dHr_dt(*inputs),dHs_dt(*inputs),landmark_ind)
    else:
        if k%3==0:
            # xs
            return construct_H(n,dHr_dxs(*inputs),dHs_dxs(*inputs),landmark_ind)
        elif k%3==1:
            # ys
            return construct_H(n,dHr_dys(*inputs),dHs_dys(*inputs),landmark_ind)
        elif k%3==2:
            # zs
            return construct_H(n,dHr_dzs(*inputs),dHs_dzs(*inputs),landmark_ind)
        
def state_transition(x, u, noise, dt=0.1):
    """state transition of a omnidirectional wheeled robot commanded by linear 
    velocity in x, y direction, and an angular velocity. In the simulation,
    it is simply implemented as two linear joints and one rotational joint

    Args:
        x (_type_): state vector. the first 3 elements correspond to robot pose
        u (_type_): control signal, vx, vy and wz
        noise (_type_): _description_
    """
    if not isinstance(noise, np.ndarray):
        noise = noise*np.ones(POSE_DIM)
    else:
        noise = np.random.multivariate_normal(mean=np.zeros(POSE_DIM),
                                              cov=noise)

    # unpack
    vx, vy, wz = u

    # update
    x[0] = x[0]+vx*dt+noise[0]
    x[1] = x[1]+vy*dt+noise[1]
    x[2] = x[2]+wz*dt+noise[2]

    return x

def state_transition_jacobian(x):
    n = x.shape[0]
    F = np.eye(n)
    G = np.zeros((n,POSE_DIM))
    G[0,0] = 1
    G[1,1] = 1
    G[2,2] = 1
    return F,G

def cam_pose_matrix(robot_pose, hc=CAM_HEIGHT):
    # unpack robot pose
    xr, yr, theta = robot_pose.flatten()

    # robot pose matrix
    Rrobot = np.array([[cos(theta), -sin(theta),0],
                       [sin(theta), cos(theta),0],
                       [0,0,1]])
    trobot = np.array([xr,yr,0])[:,np.newaxis]

    Trobot_inv = np.concatenate([Rrobot.T,-Rrobot.T.dot(trobot)],axis=1)
    Trobot_inv = np.concatenate([Trobot_inv, np.array([[0,0,0,1]])],axis=0)

    Tbc_inv = np.array([[0,-1, 0,  0],
                        [0, 0,-1, hc],
                        [1, 0, 0,  0],
                        [0, 0, 0,  1]])
    
    Tcam_inv = multi_dot([Tbc_inv, Trobot_inv])
    return Tcam_inv

def get_cam_frame_coord(scene_pt, Tcam_inv):
    xs,ys,zs = scene_pt.flatten()
    h_coord = np.array([xs,ys,zs,1])[:,np.newaxis]
    xs_hat, ys_hat, zs_hat, _ = Tcam_inv.dot(h_coord).flatten()
    return xs_hat, ys_hat, zs_hat

def get_world_coord(scene_pt, Tcam_inv):
    xs_hat,ys_hat,zs_hat = scene_pt.flatten()
    h_coord = np.array([xs_hat,ys_hat,zs_hat,1])[:,np.newaxis]
    xs, ys, zs, _ = matinv(Tcam_inv).dot(h_coord).flatten()
    return xs, ys, zs

def project(xs_hat, ys_hat, zs_hat, fx=FX,fy=FY,px=PX,py=PY):
    ux = fx*xs_hat/zs_hat+px
    uy = fx*ys_hat/zs_hat+py
    return ux, uy

def observation_model(robot_pose,scene_pt):
    """ get the camera projection points of the landmarks

    Args:
        x (_type_): _description_
        camera_height (_type_): _description_
        calibration_matrix (_type_): _description_
    """
    if isinstance(robot_pose, list):
        robot_pose = np.array(robot_pose)
    if isinstance(scene_pt, list):
        scene_pt = np.array(scene_pt)
    Tcam_inv = cam_pose_matrix(robot_pose)
    xs_hat, ys_hat, zs_hat = get_cam_frame_coord(scene_pt, Tcam_inv)
    ux,uy = project(xs_hat, ys_hat, zs_hat)

    return ux, uy

def loss_func(vars,x0,P0):
    """the loss function to be minimized by the scipy.optimize.minimize function
    vars consist of a series of both state x and control input u. 

    Args:
        vars (_type_): _description_
    """
    Q = PROCESS_NOISE_COV # 3 by 3
    R = OBS_NOISE_COV # 2 by 2
    dt = DELTA_T

    x_dim = x0.shape[0]
    u_dim = POSE_DIM
    n_landmark = int((x_dim-POSE_DIM)/SCENE_PT_DIM)
    
    ########################### 1. FORWARD PASS ###############################

    # unpack vars, supposing ut,xt,ut+1,xt+1
    dim = u_dim+x_dim
    n_pass = int(vars.shape[0]/dim)
    x_prev = x0
    P_prev = P0
    cache = [None]*n_pass
    for i in range(n_pass):
        # the jacobians are evaluated at the series of x's
        u = vars[i*dim:i*dim+u_dim]
        x = vars[i*dim+u_dim:(i+1)*dim]

        # evaluate jacobian w.r.t state at the previous state
        F,G = state_transition_jacobian(x_prev)

        x_pred = state_transition(x_prev, u, 0) # zero noise
        x_next = x_pred
        # y_pred = observation_model(x_pred)
        # the covariance matrix after motion model / prediction step
        P_pred = multi_dot([F,P_prev,F.T]) + \
            multi_dot([G,Q,G.T])

        # set P_ to P_pred to iterate through all landmarks
        P_ = P_pred
        cache_pred = [None]*n_landmark # cache for storing intermediate P_'s

        # go through each landmark
        for ind in range(n_landmark):
            # jacobian of measurement model w.r.t state
            H = observation_jacobian(x_pred, landmark_ind=ind)

            S = multi_dot([H, P_, H.T])+R

            # Kalman gain
            K = multi_dot([P_, H.T, matinv(S)])

            # update cov P
            I_KH = np.eye(x_dim)-K.dot(H)
            P_next = I_KH.dot(P_)

            cache_pred[ind]=(P_,P_next,K,H,I_KH)
            P_ = P_next
        
        cache[i] = (x_prev,x_next,F,G,P_prev,P_pred,P_next,cache_pred,Q,R)

        # update variables
        x_prev = x # note this is the variable to be optimized over
        P_prev = P_next

    P0_inv = np.linalg.inv(P0)
    loss = np.trace(P0_inv.dot(P_next))

    ########################### 2. BACKWARD PASS ##############################
    
    # ref: Benhamou, J., Bonnabel, S., & Chapdelaine, C. (2024). 
    # Backpropagation-Based Analytical Derivatives of EKF Covariance for Active 
    # Sensing. arXiv preprint arXiv:2402.17569. 

    # initialize gradient w.r.t PN
    dL_dPn = P0_inv 

    # calculate derivative w.r.t the matrices
    dmat = [None]*n_pass
    for i in np.arange(n_pass-1,-1,-1):
        # unpack cache
        x_prev,x_next,Fn,Gn,Pnm1,Pn_nm1,Pn,cache_pred,Q,R = cache[i]

        # dL/dP(n-1)|(n-1)
        dL_dPn_list = [None]*n_landmark
        dL_dPn_list[-1] = dL_dPn
        dL_dHn_list = [None]*n_landmark
        for j in np.arange(n_landmark-1,-1,-1):
            P_,P_next,K,H,I_KH = cache_pred[j]
            if j>0:
                tmp = dL_dPn_list[j]
                dL_dPn_list[j-1] = multi_dot([I_KH.T,dL_dPn_list[j],I_KH])
            else: # j=0
                # dL/dPn|(n-1)
                dL_dPn_nm1 = multi_dot([I_KH.T,dL_dPn_list[0],I_KH])

            # dL/dHn
            dL_dHn_list[j] = np.transpose(-2*multi_dot(
                [P_next, dL_dPn_list[j], P_next, H.T, matinv(R)]))

        dL_dPnm1 = multi_dot([Fn.T,dL_dPn_nm1,Fn])

        # dL/dFn
        dL_dFn = 2*multi_dot([dL_dPn_nm1, Fn, Pnm1])

        # dL/dGn
        dL_dGn = 2*multi_dot([dL_dPn_nm1, Gn, Q])

        # dL/dRn
        dL_dRn = None # the paper calculates this term, but our implementation 
                      # does not have Rn depending on x

        # append all results
        dmat[i]=(dL_dPn_list,dL_dPn_nm1,dL_dPnm1,dL_dFn,dL_dGn,dL_dHn_list)

        # make current
        dL_dPn = dL_dPnm1
    
    grad = None
    dL_dxn = np.zeros((x_dim,n_pass))
    dL_du = np.zeros((u_dim, n_pass ))
    for i_pass in np.arange(n_pass-1,-1,-1):
        # unpack cache
        x_prev,x_next,Fn,Gn,Pnm1,Pn_nm1,Pn,cache_pred,Q,R = cache[i_pass]
        dL_dPn_list,dL_dPn_nm1,dL_dPnm1,dL_dFn,dL_dGn,dL_dHn_list = dmat[i_pass]

        # compute gradient for x, element wise
        for ind in range(n_landmark):
            for k in range(x_dim):
                dH_dxk = dH_dxk_fcn(x_next,landmark_ind=ind,k=k)
                dL_dxn[k,i_pass] += np.trace(multi_dot([
                    dL_dHn_list[ind].T, dH_dxk
                    ]))

        # if i < n_pass-1, add the derivative coming from xn+1
        E = np.eye(x_dim)
        if i_pass < n_pass-1:
            for k in range(x_dim):
                dL_dxn[k,i_pass] += np.trace(multi_dot([
                    dL_dxn[:,[i_pass+1]].T, E[:,[k]]
                    ]))      

        # compute gradient for u, element-wise
        for k in range(u_dim):
            # first two terms in the paper are 0
            tmp = np.zeros((x_dim,1))
            tmp[k,0] = dt
            dL_du[k,i_pass] = np.trace(multi_dot([dL_dxn[:,[i_pass]].T, tmp]))

    grad = np.vstack([dL_du, dL_dxn]).T
    grad = grad.reshape((-1,1))

    return loss, grad

### Test implementation

In [20]:
# assume two scene points
robot_pose = [0,0,0]
scene_pt1 = [1.0,1.0,1.0]
scene_pt2 = [1.0,-1.0,1.0]

x0 = np.array(robot_pose+scene_pt1+scene_pt2)[:,np.newaxis]

# pick some random covariance matrix
tmp = np.random.randn(x0.shape[0],x0.shape[0])
P0 = multi_dot([tmp.T,tmp])

# suppose moving constant in one direction, no rotation
vx = 0.1
vy = 0.1
w = 0
n_pass = 3
x = x0
vars = None
for i_pass in range(n_pass):
    u = np.array([vx,vy,w])[:,np.newaxis]
    x = state_transition(x,u,0)
    
    tmp = np.concatenate([u,x],axis=0)
    if vars is None:
        vars = tmp
    else:
        vars = np.concatenate([vars,tmp],axis=0)

loss, grad = loss_func(vars,x0,P0)

print(loss)
print(grad)


[[ 0.04]
 [ 0.04]
 [ 0.  ]
 [ 1.  ]
 [ 1.  ]
 [ 1.  ]
 [ 1.  ]
 [-1.  ]
 [ 1.  ]]


### Constraint function for SLSQP

In [46]:
# Example of u_min & u_max
u_min = 0.0
u_max = 1.0

x_dim = x0.shape[0]
u_dim = POSE_DIM
dim = x_dim+u_dim

def constraint_func(vars):
    N = int(len(vars)/dim)
    print("Steps (N): ", N)
    state_cons = []
    control_cons_1 = []
    control_cons_2 = []
    
    prev_state = None

    for n in range(0, len(vars), 12):
        # print(n)
        state = vars[n:n+x_dim]
        control = vars[n+9:n+12]

        if n == 0:
            state_cons.append(state - x0)
        else:
            # state transition contraint: x_n = f(x_{n-1}, u_n, w_n)
            state_cons.append(state - state_transition(prev_state, control, 0))

        # control input constraints u_min <= u_n <= u_max
        control_cons_1.append(control-u_min) # u_min <= u_n
        control_cons_2.append(u_max-control) # u_n <= u_max

        prev_state = state

    return np.array(state_cons), np.array(control_cons_1), np.array(control_cons_2)

state_constraints, control_constraints_1,control_constraints_2 = constraint_func(vars)
# print(state_cons.shape, control_cons_2.shape)

# An example for SLSQP
# state_cons = {'type':'eq',
#               'fun': state_constraints[0]}
# control_cons_1 = {'type': 'ineq',
#                 'fun': control_constraints_1[0]}
# control_cons_2 = {'type': 'ineq',
#                 'fun': control_constraints_2[0]}

# result = minimize(objective_function {Loss(P)}, x0, method='SLSQP', constraints=[state_cons, control_cons_1, control_cons_2])

Steps (N):  3
{'type': 'eq', 'fun': array([[ 1.46],
       [-1.34],
       [ 1.4 ],
       [-0.98],
       [-0.98],
       [-1.  ],
       [ 0.  ],
       [ 2.  ],
       [ 0.  ]])}


### Simulation loop

In [33]:
# define a model to sample scene points 3D location given the pixel value
def pixel_to_3d(ux,uy,robot_pose,zs_min=0.5,zs_max=5):
    # sample a zs
    zs_hat = zs_min + np.random.rand()*(zs_max-zs_min)

    xs_hat = (ux-PX)/FX*zs_hat
    ys_hat = (uy-PY)/FY*zs_hat
    Tcam_inv = cam_pose_matrix(robot_pose)
    xs,ys,zs = get_world_coord(np.array([xs_hat,ys_hat,zs_hat]),Tcam_inv)
    return xs,ys,zs

In [42]:
# define current robot pose and ground truth scene point location
robot_pose = [0,0,0]
scene_pt1_gt = [2.0,1.0,0.0]
scene_pt2_gt = [2.0,-1.0,0.0]

# get pixel value
ux1,uy1 = observation_model(robot_pose, scene_pt1_gt)
ux2,uy2 = observation_model(robot_pose, scene_pt2_gt)
print((ux1,uy1))
print((ux2,uy2))

# get 3d location beliefs
np.random.seed(9)
pt1_bel = pixel_to_3d(ux1,uy1,np.array(robot_pose))
pt2_bel = pixel_to_3d(ux2,uy2,np.array(robot_pose))
print(pt1_bel)
print(pt2_bel)

# construct initial state
x0 = np.array(robot_pose+[*pt1_bel]+[*pt2_bel])[:,np.newaxis]

# initialize P0, large variance for the scene points
P0 = np.diag([0,0,0]+[1e6,1e6,1e6]*2)


(-0.5, 0.5)
(0.5, 0.5)
(0.5466836924856497, 0.27334184624282487, 0.7266581537571751)
(2.7584356646693244, -1.3792178323346622, -0.3792178323346622)
